In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd

In [2]:
data2001 = pd.read_csv('./pems06/d06_text_station_5min_2018_02_28.txt.gz',header=None, usecols=range(12))
# Assign column names based on the provided headers
column_names = [
    "Timestamp", "Station", "District", "Freeway #", 
    "Direction of Travel", "Lane Type", "Station Length", 
    "Samples", "% Observed", "Total Flow", "Avg Occupancy", "Avg Speed"
]

# Assign column names to the dataframe
data2001.columns = column_names
data2001

,Timestamp,Station,District,Freeway #,Direction of Travel,Lane Type,Station Length,Samples,% Observed,Total Flow,Avg Occupancy,Avg Speed
0,02/28/2018 00:00:00,601100,6,180,W,ML,0.311,10,100,2.0,0.0022,65.0
1,02/28/2018 00:00:00,601101,6,41,S,ML,0.439,20,100,6.0,0.0019,67.1
2,02/28/2018 00:00:00,601102,6,41,S,ML,2.585,20,100,6.0,0.0019,67.1
3,02/28/2018 00:00:00,601103,6,947,E,ML,0.460,20,100,10.0,0.0030,67.5
4,02/28/2018 00:00:00,601105,6,99,N,ML,0.510,30,67,44.0,0.0077,65.4
...,...,...,...,...,...,...,...,...,...,...,...,...
160699,02/28/2018 23:55:00,649952,6,99,S,FR,NaN,10,100,2.0,NaN,NaN
160700,02/28/2018 23:55:00,649953,6,99,N,FR,NaN,0,0,NaN,NaN,NaN
160701,02/28/2018 23:55:00,649961,6,99,S,FR,NaN,10,100,0.0,NaN,NaN
160702,02/28/2018 23:55:00,758410,6,58,W,ML,0.459,18,100,32.0,0.0086,65.2


In [3]:
comm = np.load('pems06_comm.npy')
comm

array([601100, 601101, 601102, 601103, 601205, 601206, 601207, 601208,
       601209, 601211, 601212, 601213, 601214, 601215, 601216, 601217,
       601218, 601219, 601220, 601221, 601222, 601225, 601226, 601227,
       601228, 601229, 601230, 601233, 601235, 601236, 601242, 601247,
       601248, 601249, 601251, 601300, 601307, 601337, 601358, 601379,
       601425, 601436, 601526, 601528, 601530, 601536, 601538, 601540,
       601542, 601612, 601929, 602100, 602101, 602102, 602103, 602205,
       602206, 602207, 602208, 602209, 602210, 602211, 602212, 602213,
       602214, 602215, 602216, 602217, 602218, 602219, 602220, 602221,
       602222, 602225, 602226, 602227, 602228, 602229, 602230, 602233,
       602235, 602236, 602238, 602242, 602247, 602248, 602249, 602251,
       602300, 602337, 602379, 602425, 602436, 602526, 602530, 602536,
       602538, 602540, 602542, 602612, 602929, 619951, 619952, 619953,
       619961, 619966, 619971, 619972, 619973, 619975, 619977, 619980,
      

In [5]:
import pandas as pd
import calendar
from datetime import date, timedelta

def read_and_process_data(file_path):
    try:
        # Read the CSV file, specifying the usecols parameter to only load the columns of interest
        data = pd.read_csv(
            file_path, 
            header=None, 
            usecols=[0, 1, 9],  # Column indexes for Timestamp, Station, and Total Flow
            names=['Timestamp', 'Station', 'Avg Flow'],  # Assigning column names
            compression='gzip'
        )

        # Convert the Timestamp column to datetime format and set it as the index
        data['Timestamp'] = pd.to_datetime(data['Timestamp'])
        data.set_index('Timestamp', inplace=True)

        # Pivot the table to get Stations as columns and Total Flow as cell values
        pivot_data = data.pivot(columns='Station', values='Avg Flow')

        return pivot_data

    except EOFError:
        print(f"Error processing file: {file_path}")
        return None

# Read the data for February 28 to get the column names (Station IDs)
# feb_28_data = read_and_process_data('./pems03/d03_text_station_5min_2018_02_28.txt.gz')
# feb_28_columns = feb_28_data.columns

# Initialize a list to hold the data for all days
all_data = []

start_date = date(2009, 12, 4)  # 开始日期
# end_date = date(2012, 10, 2)   # 结束日期
end_date = date(2024, 3, 20)   # 结束日期
current_date = start_date
all_data = []                  # 存储所有数据的列表

while current_date <= end_date:
    year = current_date.year
    month = current_date.month
    day = current_date.day
    file_path = f'./pems06/d06_text_station_5min_{year}_{month:02d}_{day:02d}.txt.gz'
#     if year==2012 and month==10 and day==1:
#         continue
    # 检查文件是否存在
    if not os.path.exists(file_path):
        print(f"File not found: {file_path}, skipping...")
        current_date += timedelta(days=1)
        continue
    
    day_data = read_and_process_data(file_path)
    
    # 使用字典存储新列，以便一次性添加到数据帧
    new_columns = {}
    for col in comm:
        if col not in day_data:
            new_columns[col] = 0  # 假设使用0填充缺失的列

    # 使用 pd.concat 添加所有新列
    if new_columns:
        new_data = pd.DataFrame(new_columns, index=day_data.index)
        day_data = pd.concat([day_data, new_data], axis=1)

    # 将列重新排序以匹配2月28日的顺序
    day_data = day_data[comm]
    
    all_data.append(day_data)
    
    current_date += timedelta(days=1)

# Concatenate all daily data into a single DataFrame
combined_data = pd.concat(all_data)

# Fill any remaining missing values with 0
combined_data.fillna(0, inplace=True)

# The combined_data DataFrame now holds the merged data for January 1 to February 28
# with columns aligned to February 28 and missing values filled with 0
# Note: The code execution is commented out to prevent execution in this environment.
# combined_data.head()
combined_data.to_csv('pems06_all_common_flow.csv')

In [ ]:
combined_data

In [1]:
import pandas as pd
pems06 = pd.read_csv('pems06_all_common_flow.csv')

# Convert the 'date' column to datetime
pems06['date'] = pd.to_datetime(pems06['date'])

# Set the 'date' column as the index
pems06.set_index('date', inplace=True)

# Resample to hourly data and aggregate using sum
pems06_hourly = pems06.resample('H').sum()

# Reset the index if you want the 'date' column back
pems06_hourly.reset_index(inplace=True)
pems06_hourly.to_csv('pems06_h.csv',index=False)


# Convert the 'date' column to datetime
pems06 = pd.read_csv('pems06_all_common_flow.csv')
pems06['date'] = pd.to_datetime(pems06['date'])

# Set the 'date' column as the index
pems06.set_index('date', inplace=True)

# Resample to hourly data and aggregate using sum
pems06_daily = pems06.resample('D').sum()

# Reset the index if you want the 'date' column back
pems06_daily.reset_index(inplace=True)
pems06_daily.to_csv('pems06_d.csv',index=False)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_52388\86321877.py:11: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  pems06_hourly = pems06.resample('H').sum()
